![](../img/dl_banner.jpg)

# 深度相似度排序模型

**提示：如果大家觉得计算资源有限，欢迎大家在翻-墙后免费试用[google的colab](https://colab.research.google.com)，有免费的K80 GPU供大家使用，大家只需要把课程的notebook上传即可运行**

#### \[稀牛学院 x 网易云课程\]《深度学习工程师(实战)》课程资料 by [@寒小阳](https://blog.csdn.net/han_xiaoyang)
当我们构建一个图像检索系统的时候，我们很核心的一个问题是，如何评估图像相似度。我们上一个notebook讲到的方式，是通过用作分类的卷积神经网络抽取图像特征，比对图像特征的距离来判断图像是否相似。

上面提到的方法当然可以完成检索任务，但是一方面这个检索过程并不是一个端到端学习出来的结果；另外一方面两张图片是否相似是一个主观性很强的东西，有一些图片，在一部分看来是相似的，在另外一部分人看来并不那么相似，我们有没有办法让神经网络通过观察标注好的相似与不相似的图片，去以数据驱动的方式总结出来如何评估相似呢？这就是我们提到的deep ranking模型，这种模型会以三元组形态的训练样本来学习细粒度的图像相似性。

## 三元组训练集
#### \[稀牛学院 x 网易云课程\]《深度学习工程师(实战)》课程资料 by [@寒小阳](https://blog.csdn.net/han_xiaoyang)
那么什么是三元组呢，三元组包含查询图像，正图像和负图像，其中正图像与查询图像比负图像更相似。下图说明了这一点：
![](../img/triplet.png)
在上图中，每列是一个三元组样本。上中下行对应于查询图像，正图像和负图像，其中正图像相比于负图像和查询图像更相似。

三元组样本可以帮助神经网络学习到相似性排序，也就是图A和图B，现对于图C和图B更接近一些。深度排序模型可以使用这种细粒度图像相似性信息，这种相对性信息在类别级别图像相似性模型或分类模型中没有办法学习到，深度排序模型在不少场景下有更贴近场景的效果。

## 图像相似度学习vs图像分类
#### \[稀牛学院 x 网易云课程\]《深度学习工程师(实战)》课程资料 by [@寒小阳](https://blog.csdn.net/han_xiaoyang)
考虑停放在道路上的三辆汽车：一辆黑色轿车，一辆白色轿车和一辆深灰色汽车。对于图像分类模型，这三款车都只是汽车。它不关心汽车的颜色和其他方面。对于类似的图像深度排序模型，它也会查看汽车的颜色和其他方面。如果查询图像是“黑色汽车”，则类似图像排序模型将“深灰色汽车”排列为高于“白色汽车”。因此，图像分类模型可能不直接适合于区分细粒度图像相似性的任务。[深度排序论文](https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/42945.pdf)的作者提出使用深度排名模型来学习细粒度图像相似性，该模型表征与一组三元组的细粒度图像相似性关系。

## 如何测量两幅图像的相似度？
#### \[稀牛学院 x 网易云课程\]《深度学习工程师(实战)》课程资料 by [@寒小阳](https://blog.csdn.net/han_xiaoyang)
用于测量图像之间相似性的度量可能是构建图像相似度模型中最重要的因素。虽然可以使用不同的度量来定义相似性，但最常用的是L1范数（也称为曼哈顿距离）和L2范数（也称为欧几里德距离）。深度排序模型在图像相似性的背景下，给出了一个非常好的解释和曼哈顿与欧几里德距离之间的比较。他们的结果表明，在我们使用的那种自然图像领域，曼哈顿距离可以更好地捕捉人类的图像相似性概念。在深度排名的情况下，作者使用Squared Euclidean距离作为相似度量。距离越小，图像越相似。

## 关于损失函数的构建
设计神经网络一个很重要的环节是构建合理的损失函数，回到任务本身，如果得到一个合理的评估相似度的深度学习模型，我们需要保证的事情是如下的公式：
$$\begin{array} { l } { D \left( f \left( p _ { i } \right) , f \left( p _ { i } ^ { + } \right) \right) < D \left( f \left( p _ { i } \right) , f \left( p _ { i } ^ { - } \right) \right) } \\ { \forall p _ { i } , p _ { i } ^ { + } , p _ { i } ^ { - } \text { such that } r \left( p _ { i } , p _ { i } ^ { + } \right) > r \left( p _ { i } , p _ { i } ^ { - } \right) } \end{array}$$

其中'f'是将图像映射到矢量的嵌入函数。$P_i$是查询图像，$P_i^+$是正图像，$P_i^-$是负图像，'r'是两个图像之间的相似距离。

参考大家在SVM中学习到的合页损失，我们可以把损失函数设计成如下的形式：
$$l \left( p _ { i } , p _ { i } ^ { + } , p _ { i } ^ { - } \right) = \max \left\{ 0 , g + D \left( f \left( p _ { i } \right) , f \left( p _ { i } ^ { + } \right) \right) - D \left( f \left( p _ { i } \right) , f \left( p _ { i } ^ { - } \right) \right) \right\}$$

其中'l'是三元组的合页损失，'g'是一个间隔超参数，它规定了两个图像对的距离之间的差距：$(P_i, P_i^+)$和$(P_i, P_i^-)$，'D'是两个欧几里德点之间的欧氏距离。

## 模型构建
#### \[稀牛学院 x 网易云课程\]《深度学习工程师(实战)》课程资料 by [@寒小阳](https://blog.csdn.net/han_xiaoyang)
我们把整个网络设计为如下的形态
![](../img/network_structure.png)

该网络由3部分组成：
- 三元组采样
- ConvNet
- 图像相似度排序

网络接受图像三元组作为输入。一个图像三元组包含查询图像$p_i$，正图像$p_i^+$和负图像$p_i^-$，它们被独立地送入到具有共享架构和参数的三个相同的深度卷积神经网络 $f(p)$中。三元组表征三个图像的相对相似关系。深度神经网络$f(p)$计算图像$p_i$的嵌入：$f(p_i)∈R_d$，其中$d$是特征嵌入的维度，$R$表示实数空间。

顶部的排序层计算三元组的合页损失。学习过程中，不断使用loss反向传播得到的梯度调整参数，减小损失。

上图中的ConvNet是一种新颖的多尺度深度神经网络结构，具体结构如下：
![](../img/CNN_structure.png)

其中上端的ConvNet是一个典型的深度卷积神经网络，抽取大部分的图像语义信息，下方的两个部分对图像下采样后使用更浅的神经网络结构用于捕捉细节纹理特征(大家指导卷积神经网络的浅层卷积层捕捉的特征会更底层) ，我们讲神经网络捕捉到的不同粒度的信息通过线性嵌入层进行组合。

## 三元组样本的选择
#### \[稀牛学院 x 网易云课程\]《深度学习工程师(实战)》课程资料 by [@寒小阳](https://blog.csdn.net/han_xiaoyang)
另外一个很重要的环节是构建样本，这里的三元组样本的质与量会直接影响模型学习的好坏，我们知道总体说来，我们要从相似的图片中选出一个pair，再从不相似的图片中抽取一张作为负样本。

![](../img/sampling.png)

注意，如果想让模型学习得更好，大家在构建负样本的时候，不仅要随机选取不同类别(图像内容不同)的图片，还要选择更有挑战一些的图片(内容有一定相似性，但是人判断相对于正样本更不相似的图片)。

In [0]:
!rm -rf tiny*
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip

    100% |████████████████████████████████| 591.8MB 30kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x60d06000 @  0x7f7d2e21c2a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
    100% |████████████████████████████████| 61kB 24.2MB/s 
    100% |████████████████████████████████| 2.0MB 3.1MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0
--2019-01-13 12:11:13--  http://cs231n.stanford.edu/tiny-imagenet-200.zip
Resolving cs231n.stanford.edu (cs231n.stanford.edu)... 171.64.68.10
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248100043 (237M) [application/zip]
Saving to: ‘tiny-imagenet-200.zip’

tiny-im

In [0]:
import zipfile
zfile = zipfile.ZipFile('tiny-imagenet-200.zip','r')
zfile.extractall()
zfile.close()

In [0]:
import os
import random
import shutil

# 递归遍历文件夹，并以一定的几率把图像名写入文件
def gci(filepath, outpath):
  #遍历filepath下所有文件，包括子目录
  files = os.listdir(filepath)
  for fi in files:
    fi_d = os.path.join(filepath,fi)            
    if os.path.isdir(fi_d):
      gci(fi_d, outpath)                  
    else:
      if random.random()<=0.05 and fi_d.endswith(".JPEG"):
        name_parts = os.path.join(fi_d).split("/")
        directory = outpath+"/"+name_parts[2]
        if not os.path.exists(directory):
          os.makedirs(directory)
        shutil.copy2(os.path.join(fi_d), directory)

In [0]:
filepath = "tiny-imagenet-200"
outpath = "triplet"
gci(filepath, outpath)

In [0]:
!ls triplet/n02963159

n02963159_114.JPEG  n02963159_293.JPEG	n02963159_394.JPEG  n02963159_446.JPEG
n02963159_121.JPEG  n02963159_297.JPEG	n02963159_39.JPEG   n02963159_466.JPEG
n02963159_136.JPEG  n02963159_304.JPEG	n02963159_3.JPEG    n02963159_467.JPEG
n02963159_158.JPEG  n02963159_318.JPEG	n02963159_402.JPEG  n02963159_480.JPEG
n02963159_177.JPEG  n02963159_326.JPEG	n02963159_418.JPEG  n02963159_49.JPEG
n02963159_226.JPEG  n02963159_33.JPEG	n02963159_420.JPEG  n02963159_52.JPEG
n02963159_233.JPEG  n02963159_352.JPEG	n02963159_439.JPEG  n02963159_58.JPEG
n02963159_234.JPEG  n02963159_371.JPEG	n02963159_43.JPEG   n02963159_5.JPEG
n02963159_279.JPEG  n02963159_388.JPEG	n02963159_440.JPEG


## 三元组采样
#### \[稀牛学院 x 网易云课程\]《深度学习工程师(实战)》课程资料 by [@寒小阳](https://blog.csdn.net/han_xiaoyang)

我们从形如下面的文件组织形式中产出三元组用于训练
```
dataset/
|__SimilarityClass1/
|    |__Image1.jpg, Image2.jpg and so on....
|
|__SimilarityClass2/
|    |_Image1.jpg, Image2.jpg and so on....
|
and so on...
```

In [0]:
import glob
import json
import random
import csv
import os
import re
import argparse
import numpy as np

def list_pictures(directory, ext='JPEG|jpg|jpeg|bmp|png|ppm'):
    return [os.path.join(root, f)
            for root, _, files in os.walk(directory) for f in files
            if re.match(r'([\w]+\.(?:' + ext + '))', f)]


def get_negative_images(all_images,image_names,num_neg_images):
    random_numbers = np.arange(len(all_images))
    np.random.shuffle(random_numbers)
    if int(num_neg_images)>(len(all_images)-1):
        num_neg_images = len(all_images)-1
    neg_count = 0
    negative_images = []
    for random_number in list(random_numbers):
        if all_images[random_number] not in image_names:
            negative_images.append(all_images[random_number])
            neg_count += 1
            if neg_count>(int(num_neg_images)-1):
                break
    return negative_images

def get_positive_images(image_name,image_names,num_pos_images):
    random_numbers = np.arange(len(image_names))
    np.random.shuffle(random_numbers)
    if int(num_pos_images)>(len(image_names)-1):
        num_pos_images = len(image_names)-1
    pos_count = 0
    positive_images = []
    for random_number in list(random_numbers):
        if image_names[random_number]!= image_name:
            positive_images.append(image_names[random_number])
            pos_count += 1 
            if int(pos_count)>(int(num_pos_images)-1):
                break
    return positive_images

def triplet_sampler(directory_path, output_path,num_neg_images,num_pos_images):
    classes = [d for d in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, d))]
    all_images = []
    for class_ in classes:
        all_images += (list_pictures(os.path.join(directory_path,class_)))
    triplets = []
    for class_ in classes:
        image_names = list_pictures(os.path.join(directory_path,class_))
        for image_name in image_names:
            image_names_set = set(image_names)
            query_image = image_name
            positive_images = get_positive_images(image_name,image_names,num_pos_images)
            for positive_image in positive_images:
                negative_images = get_negative_images(all_images,set(image_names),num_neg_images)
                for negative_image in negative_images:
                    triplets.append(query_image+',')
                    triplets.append(positive_image+',')
                    triplets.append(negative_image+'\n')
            
    f = open(os.path.join(output_path,"triplets.txt"),'w')
    f.write("".join(triplets))
    f.close()

In [0]:
input_dir = 'triplet'
output_dir = 'train_samples'
if not os.path.exists(output_dir):
  os.makedirs(output_dir)
num_neg_images = 10
num_pos_images = 10
triplet_sampler(directory_path=input_dir, output_path=output_dir, num_neg_images=num_neg_images, num_pos_images=num_pos_images)

In [0]:
!head -5 train_samples/triplets.txt

triplet/n02843684/n02843684_308.JPEG,triplet/n02843684/n02843684_230.JPEG,triplet/n02226429/n02226429_20.JPEG
triplet/n02843684/n02843684_308.JPEG,triplet/n02843684/n02843684_230.JPEG,triplet/n07720875/n07720875_376.JPEG
triplet/n02843684/n02843684_308.JPEG,triplet/n02843684/n02843684_230.JPEG,triplet/n02917067/n02917067_0.JPEG
triplet/n02843684/n02843684_308.JPEG,triplet/n02843684/n02843684_230.JPEG,triplet/n04265275/n04265275_44.JPEG
triplet/n02843684/n02843684_308.JPEG,triplet/n02843684/n02843684_230.JPEG,triplet/images/val_7166.JPEG


In [0]:
!wc -l train_samples/triplets.txt

834900 train_samples/triplets.txt


## 模型构建与训练
#### \[稀牛学院 x 网易云课程\]《深度学习工程师(实战)》课程资料 by [@寒小阳](https://blog.csdn.net/han_xiaoyang)

In [0]:
from  __future__ import absolute_import
from __future__ import print_function
from ImageDataGeneratorCustom import ImageDataGeneratorCustom
import numpy as np
from keras.applications.vgg16 import VGG16
from keras.layers import *
from keras.models import Model, load_model
from keras.optimizers import SGD
from keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf
from keras import backend as K

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
K.set_session(sess)

def convnet_model_():
    vgg_model = VGG16(weights=None, include_top=False)
    x = vgg_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.6)(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.6)(x)
    x = Lambda(lambda  x_: K.l2_normalize(x,axis=1))(x)
    convnet_model = Model(inputs=vgg_model.input, outputs=x)
    return convnet_model

def deep_rank_model():
 
    convnet_model = convnet_model_()
    first_input = Input(shape=(224,224,3))
    first_conv = Conv2D(96, kernel_size=(8, 8),strides=(16,16), padding='same')(first_input)
    first_max = MaxPool2D(pool_size=(3,3),strides = (4,4),padding='same')(first_conv)
    first_max = Flatten()(first_max)
    first_max = Lambda(lambda  x: K.l2_normalize(x,axis=1))(first_max)

    second_input = Input(shape=(224,224,3))
    second_conv = Conv2D(96, kernel_size=(8, 8),strides=(32,32), padding='same')(second_input)
    second_max = MaxPool2D(pool_size=(7,7),strides = (2,2),padding='same')(second_conv)
    second_max = Flatten()(second_max)
    second_max = Lambda(lambda  x: K.l2_normalize(x,axis=1))(second_max)

    merge_one = concatenate([first_max, second_max])

    merge_two = concatenate([merge_one, convnet_model.output])
    emb = Dense(4096)(merge_two)
    l2_norm_final = Lambda(lambda  x: K.l2_normalize(x,axis=1))(emb)

    final_model = Model(inputs=[first_input, second_input, convnet_model.input], outputs=l2_norm_final)

    return final_model


deep_rank_model = deep_rank_model()

for layer in deep_rank_model.layers:
    print (layer.name, layer.output_shape)

model_path = "./deep_ranking"

class DataGenerator(object):
    def __init__(self, params, target_size=(224, 224)):
        self.params = params
        self.target_size = target_size
        self.idg = ImageDataGeneratorCustom(**params)

    def get_train_generator(self, batch_size):
        return self.idg.flow_from_directory("./triplet",
                                            batch_size=batch_size,
                                            target_size=self.target_size,
                                            shuffle=False,
                                            triplet_path  ='./train_samples/triplets.txt'
                                           )

    def get_test_generator(self, batch_size):
        return self.idg.flow_from_directory("./triplet",
                                            batch_size=batch_size,
                                            target_size=self.target_size, 
                                            shuffle=False,
                                            triplet_path  ='./train_samples/triplets.txt'
                                        )



dg = DataGenerator({
    "rescale": 1. / 255,
    "horizontal_flip": True,
    "vertical_flip": True,
    "zoom_range": 0.2,
    "shear_range": 0.2,
    "rotation_range": 30,
"fill_mode": 'nearest' 
}, target_size=(224, 224))

batch_size = 8 
batch_size *= 3
train_generator = dg.get_train_generator(batch_size)


_EPSILON = K.epsilon()
def _loss_tensor(y_true, y_pred):
    y_pred = K.clip(y_pred, _EPSILON, 1.0-_EPSILON)
    loss =  tf.convert_to_tensor(0,dtype=tf.float32)
    g = tf.constant(1.0, shape=[1], dtype=tf.float32)
    for i in range(0,batch_size,3):
        try:
            q_embedding = y_pred[i+0]
            p_embedding =  y_pred[i+1]
            n_embedding = y_pred[i+2]
            D_q_p =  K.sqrt(K.sum((q_embedding - p_embedding)**2))
            D_q_n = K.sqrt(K.sum((q_embedding - n_embedding)**2))
            loss = (loss + g + D_q_p - D_q_n )            
        except:
            continue
    loss = loss/(batch_size/3)
    zero = tf.constant(0.0, shape=[1], dtype=tf.float32)
    return tf.maximum(loss,zero)

#deep_rank_model.load_weights('deepranking.h5')
deep_rank_model.compile(loss=_loss_tensor, optimizer=SGD(lr=0.001, momentum=0.9, nesterov=True))


train_steps_per_epoch = int((834900)/batch_size)
train_epocs = 5
deep_rank_model.fit_generator(train_generator,
                        steps_per_epoch=train_steps_per_epoch,
                        epochs=train_epocs,
                        verbose=1
                        )

model_path = "deepranking.h5"
deep_rank_model.save_weights(model_path)
#f = open('deepranking.json','w')
#f.write(deep_rank_model.to_json())
#f.close()

input_22 (None, None, None, 3)
block1_conv1 (None, None, None, 64)
block1_conv2 (None, None, None, 64)
block1_pool (None, None, None, 64)
block2_conv1 (None, None, None, 128)
block2_conv2 (None, None, None, 128)
block2_pool (None, None, None, 128)
block3_conv1 (None, None, None, 256)
block3_conv2 (None, None, None, 256)
block3_conv3 (None, None, None, 256)
block3_pool (None, None, None, 256)
block4_conv1 (None, None, None, 512)
block4_conv2 (None, None, None, 512)
block4_conv3 (None, None, None, 512)
block4_pool (None, None, None, 512)
block5_conv1 (None, None, None, 512)
block5_conv2 (None, None, None, 512)
block5_conv3 (None, None, None, 512)
block5_pool (None, None, None, 512)
input_23 (None, 224, 224, 3)
input_24 (None, 224, 224, 3)
global_average_pooling2d_8 (None, 512)
conv2d_15 (None, 14, 14, 96)
conv2d_16 (None, 7, 7, 96)
dense_22 (None, 4096)
max_pooling2d_15 (None, 4, 4, 96)
max_pooling2d_16 (None, 4, 4, 96)
dropout_15 (None, 4096)
flatten_15 (None, 1536)
flatten_16 (None, 15

## 预测与相似度排序
#### \[稀牛学院 x 网易云课程\]《深度学习工程师(实战)》课程资料 by [@寒小阳](https://blog.csdn.net/han_xiaoyang)

In [0]:
import argparse
import os

# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-m", "--model", required=True,
    help="Path to the deep ranking model")

ap.add_argument("-i1", "--image1", required=True,
    help="Path to the first image")

ap.add_argument("-i2", "--image2", required=True,
    help="Path to the second image")

args = vars(ap.parse_args())

if not os.path.exists(args['model']):
    print "The model path doesn't exist!"
    exit()

if not os.path.exists(args['image1']):
    print "The image 1 path doesn't exist!"
    exit()

if not os.path.exists(args['image2']):
    print "The image 2 path doesn't exist!"
    exit()

args = vars(ap.parse_args())

import numpy as np
from keras.applications.vgg16 import VGG16
from keras.layers import *
from keras.models import Model
from keras.preprocessing.image import load_img, img_to_array
from skimage import transform
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Embedding

def convnet_model_():
    vgg_model = VGG16(weights=None, include_top=False)
    x = vgg_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.6)(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.6)(x)
    x = Lambda(lambda  x_: K.l2_normalize(x,axis=1))(x)
    convnet_model = Model(inputs=vgg_model.input, outputs=x)
    return convnet_model

def deep_rank_model():
 
    convnet_model = convnet_model_()
    first_input = Input(shape=(224,224,3))
    first_conv = Conv2D(96, kernel_size=(8, 8),strides=(16,16), padding='same')(first_input)
    first_max = MaxPool2D(pool_size=(3,3),strides = (4,4),padding='same')(first_conv)
    first_max = Flatten()(first_max)
    first_max = Lambda(lambda  x: K.l2_normalize(x,axis=1))(first_max)

    second_input = Input(shape=(224,224,3))
    second_conv = Conv2D(96, kernel_size=(8, 8),strides=(32,32), padding='same')(second_input)
    second_max = MaxPool2D(pool_size=(7,7),strides = (2,2),padding='same')(second_conv)
    second_max = Flatten()(second_max)
    second_max = Lambda(lambda  x: K.l2_normalize(x,axis=1))(second_max)

    merge_one = concatenate([first_max, second_max])

    merge_two = concatenate([merge_one, convnet_model.output])
    emb = Dense(4096)(merge_two)
    l2_norm_final = Lambda(lambda  x: K.l2_normalize(x,axis=1))(emb)

    final_model = Model(inputs=[first_input, second_input, convnet_model.input], outputs=l2_norm_final)

    return final_model


model = deep_rank_model()

# for layer in model.layers:
#     print (layer.name, layer.output_shape)

model_path = 
image1_path = 
image2_path = 

model.load_weights(model_path)

image1 = load_img(image1_path)
image1 = img_to_array(image1).astype("float64")
image1 = transform.resize(image1, (224, 224))
image1 *= 1. / 255
image1 = np.expand_dims(image1, axis = 0)

embedding1 = model.predict([image1, image1, image1])[0]

image2 = load_img(image2_path)
image2 = img_to_array(image2).astype("float64")
image2 = transform.resize(image2, (224, 224))
image2 *= 1. / 255
image2 = np.expand_dims(image2, axis = 0)

embedding2 = model.predict([image2,image2,image2])[0]

distance = sum([(embedding1[idx] - embedding2[idx])**2 for idx in range(len(embedding1))])**(0.5)

print (distance)

![](../img/xiniu_neteasy.png)